# Workbook for Solving a TSP
- This is the notebook that I started developing from April 30.
- It has been constanly updated.

In [1]:
import veroviz as vrv

In [2]:
vrv.checkVersion()

'Your current installed version of veroviz is 0.4.5. You are up-to-date with the latest available version.'

In [3]:
import os
ORS_API_KEY=os.environ['ORSKEY']

# 1 -- Create some "nodes"

In [4]:
myBoundingRegion = [[43.01770145196991, -78.87840270996095], [42.878480017739044, -78.8756561279297], [42.83569550641454, -78.68270874023439], [42.96596996868038, -78.60717773437501], [43.04430359661548, -78.72528076171876]]
myBoundingRegion

[[43.01770145196991, -78.87840270996095],
 [42.878480017739044, -78.8756561279297],
 [42.83569550641454, -78.68270874023439],
 [42.96596996868038, -78.60717773437501],
 [43.04430359661548, -78.72528076171876]]

In [5]:
nodesDF = vrv.generateNodes(nodeType = 'customer',
                            nodeName = 'cust',
                            numNodes = 10,
                            incrementName = True,
                            nodeDistrib = 'uniformBB',
                            nodeDistribArgs = {'boundingRegion': myBoundingRegion},
                            snapToRoad = True,
                            dataProvider     = 'ORS-online',
                            dataProviderArgs = {'APIkey' : ORS_API_KEY})
nodesDF

,id,lat,lon,altMeters,nodeName,nodeType,popupText,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText,elevMeters
0,1,42.989309,-78.807882,0,cust1,customer,1,glyphicon,info-sign,blue,1,pin,blue,1,None
1,2,42.895772,-78.831786,0,cust2,customer,2,glyphicon,info-sign,blue,2,pin,blue,2,None
2,3,42.993455,-78.766487,0,cust3,customer,3,glyphicon,info-sign,blue,3,pin,blue,3,None
3,4,42.931355,-78.815737,0,cust4,customer,4,glyphicon,info-sign,blue,4,pin,blue,4,None
4,5,43.031264,-78.788471,0,cust5,customer,5,glyphicon,info-sign,blue,5,pin,blue,5,None
5,6,42.989209,-78.824086,0,cust6,customer,6,glyphicon,info-sign,blue,6,pin,blue,6,None
6,7,42.945878,-78.665481,0,cust7,customer,7,glyphicon,info-sign,blue,7,pin,blue,7,None
7,8,42.985002,-78.791676,0,cust8,customer,8,glyphicon,info-sign,blue,8,pin,blue,8,None
8,9,42.867562,-78.751835,0,cust9,customer,9,glyphicon,info-sign,blue,9,pin,blue,9,None
9,10,42.972128,-78.742026,0,cust10,customer,10,glyphicon,info-sign,blue,10,pin,blue,10,None


In [6]:
len(nodesDF)

10

In [7]:
# View our nodes and bounding region:
vrv.createLeaflet(nodes = nodesDF, boundingRegion=myBoundingRegion)

In [8]:
# 2. Get time and distance matrices
[timeSec, distMeters] = vrv.getTimeDist2D(nodes = nodesDF,
                                          routeType = 'euclidean2D',
                                          speedMPS = vrv.convertSpeed(25, 'miles', 'hr', 'meters', 'second'))

In [9]:
#timeSec
timeSec[1,2]

946.0241936632211

In [10]:
#distance
distMeters[1,2]

10572.740109930335

In [11]:
# Optional, just to view the data in a table:
vrv.convertMatricesDictionaryToDataframe(timeSec)

,1,2,3,4,5,6,7,8,9,10
1,0.000000,946.024194,304.862580,578.924680,440.430353,118.250711,1125.599383,125.775948,1277.561796,510.082184
2,946.024194,0.000000,1081.764347,372.631324,1383.449046,930.484985,1313.061474,934.085191,648.236007,1002.936478
3,304.862580,1081.764347,0.000000,714.369703,408.617753,422.420742,875.950697,202.108019,1255.969488,277.149397
4,578.924680,372.631324,714.369703,0.000000,1012.864644,578.304565,1106.827546,561.454374,787.515985,673.755734
5,440.430353,1383.449046,408.617753,1012.864644,0.000000,492.197137,1235.284122,460.453724,1649.079280,678.505564
6,118.250711,930.484985,422.420742,578.304565,492.197137,0.000000,1235.328166,240.184498,1319.350081,622.507981
7,1125.599383,1313.061474,875.950697,1106.827546,1235.284122,1235.328166,0.000000,999.970275,1002.088139,616.765332
8,125.775948,934.085191,202.108019,561.454374,460.453724,240.184498,999.970275,0.000000,1203.106725,384.309461
9,1277.561796,648.236007,1255.969488,787.515985,1649.079280,1319.350081,1002.088139,1203.106725,0.000000,1041.872187
10,510.082184,1002.936478,277.149397,673.755734,678.505564,622.507981,616.765332,384.309461,1041.872187,0.000000


In [12]:
def tsp_cost(route, costDict):
    cost = 0
    
    i = route[0]
    for j in route[1:]:
        cost += costDict[i,j]
        i = j
        
    cost += costDict[i, route[0]]
    
    return cost

In [13]:
def tsp_neighbor(route):
    """
        This function computes a "subtour" solution to a TSP.
        
        Inputs
        ------
        route: list of Integer, indicating the node where the salesperson begins (and ends) the route
        
        Returns
        -------
        A random list of nodeIDs specifying another TSP route.
        """
    #import random
    from random import randint 
    
    #a = random.randint(0,len(route)-3)
    a = randint(0,len(route)-3)
    #b = random.randint(a+1, len(route)-2)
    b = randint(a+1, len(route)-2)
    
    newRoute = []
    newRoute.extend(route[0:a])
    
    subtour = route[a:b+1]
    subtour.reverse()
    newRoute.extend(subtour)
    
    newRoute.extend(route[b+1:len(route)-1])
    
    newRoute.append(newRoute[0])
    
    return newRoute

In [14]:
#Example of subtour
a=[1,5,8,9,2,4,3,7,6]
tsp_neighbor(a)

[1, 5, 8, 9, 2, 3, 4, 7, 1]

In [15]:
def solve_tsp_nn(startNode, costDict, nodesDF): 
    """
    This function computes a "nearest neighbor" solution to a TSP.
    
    Inputs
    ------
    startNode: Integer, indicating the node where the salesperson begins (and ends) the route
    
    costDict: VeRoViz time or distance dictionary.
    
    nodesDF: VeRoViz nodes dataframe
    
    Returns
    -------
    An ordered list of nodeIDs specifying a TSP route.
    """
    
    # Solve the TSP with a "nearest neighbor" heuristic
    nn_route = []

    # Start our route by visiting the startNode
    nn_route.append(startNode)

    # Initialize a list of unvisited nodes
    unvisitedNodes = list(nodesDF[nodesDF['id'] != startNode]['id'])

    # Let i represent our "current" location:
    i = startNode

    while len(unvisitedNodes) > 0:
        # Initialize minTime to a huge value
        minTime = float('inf')

        # Find the nearest unvisited node to our current node:
        for j in unvisitedNodes:
            if (costDict[i,j] < minTime):
                nextNode = j
                minTime = costDict[i,j]

        # Update our salesperson's location
        i = nextNode

        # Append nextNode to our route:
        nn_route.append(nextNode)

        # Remove nextNode from our list of unvisitedNodes:
        unvisitedNodes.remove(nextNode)

    nn_route.append(startNode)

    return nn_route    

In [16]:
def solveTSP_SA(nodesDF, costDict, timeLimit):
    import time
    from random import random
    import math as m
    
    runTime=0
    start_time = time.time()
    startnode=1
    T0=4000
    I=3
    d=5
    Tfinal=2020
    cutoffTime=timeLimit
    
    # Initialize by calling your nearest neighbor function
    X0 = solve_tsp_nn(startnode,costDict,nodesDF)
    Z=tsp_cost(X0,costDict)
    print('Intial Solution : X0 =',X0,'Z0 =',Z)
    
    # Simulated annealing (SA) procedure
    Xcur = tsp_neighbor(X0)
    Zcur = tsp_cost(Xcur,costDict)    
    Tcur = T0
    print('Current Solution : Xcur =',Xcur,'Zcur =',Zcur)
    Xbest = Xcur
    Zbest = Zcur
    print('Best known Solution : Xbest =',Xbest,'Zbest =',Zbest)
    
    def solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime):
        print('Tcur=',Tcur)
        for i in range(1,I+1):
            #Generate a neighbor solution, Xcount
            # call tsp_neighbor(route)
            # ...
            # ...
            Xcount=tsp_neighbor(X0)
            Z = tsp_cost(Xcount,costDict)
            print('i=',i)
            print('Xcount=',Xcount,'Zcount=',Z)

            if (Z<Zcur):
                print('Z-Zcur=',Z-Zcur)
                Xcur = Xcount
                Zcur = Z
                print('Xcur=',Xcur,'Zcur=',Z)


            else:
                C=Z-Zcur
                print('C=',C)
                if(random()<=m.exp(-C/Tcur)):
                    Xcur = Xcount
                    Zcur = Z
                    print('Xcur=',Xcur,'Zcur=',Zcur)


            if(Z < Zbest):
                print('Z-Zbest=',Z-Zbest)
                Zbest = Z
                Xbest = Xcur
                print('Xbest=',Xbest,'Zbest=',Zbest)

        Tcur=Tcur-d
        #print('Tcur=',Tcur)
        print('Xbest=',Xbest,'Zbest=',Zbest)
            
        runTime=time.time()-start_time
        print(runTime)
        if(Tcur<Tfinal or runTime>=timeLimit):
            if(Tcur<Tfinal):
                print('Tcur=',Tcur,'< Tfinal=',Tfinal)
                print('runTime =',runTime,',cutoffTime =',cutoffTime)
            elif(runTime>=cutoffTime):
                print('runTime =',runTime,'> cutoffTime =',cutoffTime)
                print('Tcur=',Tcur,',Tfinal=',Tfinal)
            return Xbest

        return solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime)
    
    
    
    XBEST=solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime)
    
    totalruntime=time.time()-start_time
    print('totaltime=',totalruntime)
    
    # SA produces a "best" solution, X_best.
    # X_best is just a sequence of node numbers,
    # starting from and returning to a "home" location.
    # For example, Xbest = [1, 5, 3, 4, 2, 1].
    
    # Your function should return a VeRoViz "assignments" dataframe.
    # Fortunately, VeRoViz provides a function to make this very easy:
    assignmentsDF = vrv.createAssignmentsFromNodeSeq2D(
        nodeSeq          = XBEST,
        nodes            = nodesDF,
        objectID         = 'Blue Car',
        modelFile        = 'veroviz/models/car_blue.gltf',
        modelScale       = 100,
        modelMinPxSize   = 75,
        routeType        = 'fastest',
        speedMPS         = None,
        leafletColor     = 'blue',
        leafletWeight    = 3,
        leafletStyle     = 'dashed',
        leafletOpacity   = 0.8,
        useArrows        = True,
        cesiumColor      = 'blue',
        cesiumWeight     = 3,
        cesiumStyle      = 'solid',
        cesiumOpacity    = 0.8,
        dataProvider     = 'ORS-online',
        dataProviderArgs = {'APIkey' : ORS_API_KEY})

    # See https://veroviz.org/docs/veroviz.createAssignments.html#veroviz.createAssignments.createAssignmentsFromNodeSeq2D
    # for more info on this function.
    
    return assignmentsDF
    #return Xbest

In [17]:
#%time
X_best=solveTSP_SA(nodesDF,timeSec,20)
X_best

Intial Solution : X0 = [1, 6, 8, 3, 10, 7, 9, 2, 4, 5, 1] Z0 = 4930.708423420916
Current Solution : Xcur = [10, 3, 8, 6, 1, 7, 9, 2, 4, 5, 10] Zcur = 5677.617685171651
Best known Solution : Xbest = [10, 3, 8, 6, 1, 7, 9, 2, 4, 5, 10] Zbest = 5677.617685171651
Tcur= 4000
i= 1
Xcount= [1, 6, 8, 5, 4, 2, 9, 7, 10, 3, 1] Zcount= 5053.486355007676
Z-Zcur= -624.1313301639748
Xcur= [1, 6, 8, 5, 4, 2, 9, 7, 10, 3, 1] Zcur= 5053.486355007676
Z-Zbest= -624.1313301639748
Xbest= [1, 6, 8, 5, 4, 2, 9, 7, 10, 3, 1] Zbest= 5053.486355007676
i= 2
Xcount= [1, 6, 8, 9, 7, 10, 3, 2, 4, 5, 1] Zcount= 6365.235469578691
C= 1311.7491145710146
Xcur= [1, 6, 8, 9, 7, 10, 3, 2, 4, 5, 1] Zcur= 6365.235469578691
i= 3
Xcount= [1, 9, 7, 10, 3, 8, 6, 2, 4, 5, 1] Zcount= 6372.268486613537
C= 7.033017034846125
Xcur= [1, 9, 7, 10, 3, 8, 6, 2, 4, 5, 1] Zcur= 6372.268486613537
Xbest= [1, 6, 8, 5, 4, 2, 9, 7, 10, 3, 1] Zbest= 5053.486355007676
0.005174160003662109
Tcur= 3995
i= 1
Xcount= [1, 6, 8, 3, 10, 7, 9, 4, 2, 5, 1] 

Xcount= [1, 9, 7, 10, 3, 8, 6, 2, 4, 5, 1] Zcount= 6372.268486613537
C= 1213.7221031825284
Xcur= [1, 9, 7, 10, 3, 8, 6, 2, 4, 5, 1] Zcur= 6372.268486613537
i= 2
Xcount= [1, 6, 8, 3, 10, 7, 9, 2, 5, 4, 1] Zcount= 6080.020473177777
Z-Zcur= -292.2480134357602
Xcur= [1, 6, 8, 3, 10, 7, 9, 2, 5, 4, 1] Zcur= 6080.020473177777
i= 3
Xcount= [1, 6, 8, 3, 4, 2, 9, 7, 10, 5, 1] Zcount= 5033.569649238607
Z-Zcur= -1046.4508239391698
Xcur= [1, 6, 8, 3, 4, 2, 9, 7, 10, 5, 1] Zcur= 5033.569649238607
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
0.22079014778137207
Tcur= 3870
i= 1
Xcount= [1, 6, 8, 3, 9, 7, 10, 2, 4, 5, 1] Zcount= 6264.228985093774
C= 1230.6593358551672
Xcur= [1, 6, 8, 3, 9, 7, 10, 2, 4, 5, 1] Zcur= 6264.228985093774
i= 2
Xcount= [1, 6, 8, 7, 10, 3, 9, 2, 4, 5, 1] Zcount= 5982.452027577094
Z-Zcur= -281.77695751668034
Xcur= [1, 6, 8, 7, 10, 3, 9, 2, 4, 5, 1] Zcur= 5982.452027577094
i= 3
Xcount= [3, 8, 6, 1, 10, 7, 9, 2, 4, 5, 3] Zcount= 5131.828610044533
Z-Zcur= -85

Xcount= [1, 6, 8, 3, 10, 9, 7, 2, 4, 5, 1] Zcount= 6020.640745153454
Z-Zcur= -30.435046393384255
Xcur= [1, 6, 8, 3, 10, 9, 7, 2, 4, 5, 1] Zcur= 6020.640745153454
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
0.8363919258117676
Tcur= 3780
i= 1
Xcount= [1, 6, 8, 3, 7, 10, 9, 2, 4, 5, 1] Zcount= 5569.293770880358
Z-Zcur= -451.34697427309584
Xcur= [1, 6, 8, 3, 7, 10, 9, 2, 4, 5, 1] Zcur= 5569.293770880358
i= 2
Xcount= [1, 6, 8, 3, 10, 5, 4, 2, 9, 7, 1] Zcount= 5677.61768517165
C= 108.32391429129166
Xcur= [1, 6, 8, 3, 10, 5, 4, 2, 9, 7, 1] Zcur= 5677.61768517165
i= 3
Xcount= [1, 6, 8, 3, 10, 7, 9, 5, 4, 2, 1] Zcount= 6437.145536936257
C= 759.5278517646066
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
0.8377280235290527
Tcur= 3775
i= 1
Xcount= [1, 6, 8, 3, 10, 7, 5, 4, 2, 9, 1] Zcount= 6001.035849271853
C= 323.4181641002033
Xcur= [1, 6, 8, 3, 10, 7, 5, 4, 2, 9, 1] Zcur= 6001.035849271853
i= 2
Xcount= [1, 7, 10, 3, 8, 6, 9, 2, 4, 5, 1] Zcount= 6255.31

Xcount= [1, 6, 7, 10, 3, 8, 9, 2, 4, 5, 1] Zcount= 6126.870677487146
Z-Zcur= -137.35830760662793
Xcur= [1, 6, 7, 10, 3, 8, 9, 2, 4, 5, 1] Zcur= 6126.870677487146
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
1.3283600807189941
Tcur= 3600
i= 1
Xcount= [1, 6, 8, 3, 10, 7, 2, 9, 4, 5, 1] Zcount= 5656.566419123506
Z-Zcur= -470.30425836363975
Xcur= [1, 6, 8, 3, 10, 7, 2, 9, 4, 5, 1] Zcur= 5656.566419123506
i= 2
Xcount= [1, 6, 7, 10, 3, 8, 9, 2, 4, 5, 1] Zcount= 6126.870677487146
C= 470.30425836363975
Xcur= [1, 6, 7, 10, 3, 8, 9, 2, 4, 5, 1] Zcur= 6126.870677487146
i= 3
Xcount= [1, 6, 8, 3, 10, 7, 9, 4, 2, 5, 1] Zcount= 5440.572803552601
Z-Zcur= -686.2978739345454
Xcur= [1, 6, 8, 3, 10, 7, 9, 4, 2, 5, 1] Zcur= 5440.572803552601
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
1.330461025238037
Tcur= 3595
i= 1
Xcount= [1, 6, 8, 3, 10, 7, 4, 2, 9, 5, 1] Zcount= 5671.662466144417
C= 231.0896625918158
Xcur= [1, 6, 8, 3, 10, 7, 4, 2, 9, 5, 1] Zcur= 5671.6624

Xcount= [1, 6, 5, 4, 2, 9, 7, 10, 3, 8, 1] Zcount= 4868.066657535071
Z-Zcur= -572.5061460175302
Xcur= [1, 6, 5, 4, 2, 9, 7, 10, 3, 8, 1] Zcur= 4868.066657535071
i= 3
Xcount= [1, 7, 10, 3, 8, 6, 9, 2, 4, 5, 1] Zcount= 6255.319037240091
C= 1387.2523797050208
Xcur= [1, 7, 10, 3, 8, 6, 9, 2, 4, 5, 1] Zcur= 6255.319037240091
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
1.52944016456604
Tcur= 3435
i= 1
Xcount= [1, 3, 8, 6, 10, 7, 9, 2, 4, 5, 1] Zcount= 5462.678877107821
Z-Zcur= -792.6401601322705
Xcur= [1, 3, 8, 6, 10, 7, 9, 2, 4, 5, 1] Zcur= 5462.678877107821
i= 2
Xcount= [4, 2, 9, 7, 10, 3, 8, 6, 1, 5, 4] Zcount= 4930.708423420916
Z-Zcur= -531.9704536869049
Xcur= [4, 2, 9, 7, 10, 3, 8, 6, 1, 5, 4] Zcur= 4930.708423420916
i= 3
Xcount= [1, 6, 8, 3, 10, 4, 2, 9, 7, 5, 1] Zcount= 5210.11830340614
C= 279.4098799852236
Xcur= [1, 6, 8, 3, 10, 4, 2, 9, 7, 5, 1] Zcur= 5210.11830340614
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
1.5306119918823242
Tcur= 3

...

Xcur= [4, 2, 9, 7, 10, 3, 8, 6, 1, 5, 4] Zcur= 4930.708423420916
i= 2
Xcount= [1, 6, 8, 3, 9, 7, 10, 2, 4, 5, 1] Zcount= 6264.228985093774
C= 1333.520561672858
Xcur= [1, 6, 8, 3, 9, 7, 10, 2, 4, 5, 1] Zcur= 6264.228985093774
i= 3
Xcount= [1, 6, 8, 9, 7, 10, 3, 2, 4, 5, 1] Zcount= 6365.235469578691
C= 101.00648448491665
Xcur= [1, 6, 8, 9, 7, 10, 3, 2, 4, 5, 1] Zcur= 6365.235469578691
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
3.4019370079040527
Tcur= 2610
i= 1
Xcount= [1, 6, 8, 3, 10, 7, 4, 2, 9, 5, 1] Zcount= 5671.662466144417
Z-Zcur= -693.5730034342741
Xcur= [1, 6, 8, 3, 10, 7, 4, 2, 9, 5, 1] Zcur= 5671.662466144417
i= 2
Xcount= [1, 6, 8, 3, 10, 5, 4, 2, 9, 7, 1] Zcount= 5677.61768517165
C= 5.955219027233397
Xcur= [1, 6, 8, 3, 10, 5, 4, 2, 9, 7, 1] Zcur= 5677.61768517165
i= 3
Xcount= [10, 3, 8, 6, 1, 7, 9, 2, 4, 5, 10] Zcount= 5677.617685171651
C= 9.094947017729282e-13
Xcur= [10, 3, 8, 6, 1, 7, 9, 2, 4, 5, 10] Zcur= 5677.617685171651
Xbest= [1, 6, 8, 4, 2, 9, 7

C= 1210.742630086098
i= 2
Xcount= [1, 6, 8, 3, 10, 7, 9, 5, 4, 2, 1] Zcount= 6437.145536936257
C= 1383.6591819285804
Xcur= [1, 6, 8, 3, 10, 7, 9, 5, 4, 2, 1] Zcur= 6437.145536936257
i= 3
Xcount= [1, 6, 8, 3, 10, 7, 5, 4, 2, 9, 1] Zcount= 6001.035849271853
Z-Zcur= -436.1096876644033
Xcur= [1, 6, 8, 3, 10, 7, 5, 4, 2, 9, 1] Zcur= 6001.035849271853
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
3.841719150543213
Tcur= 2400
i= 1
Xcount= [1, 6, 8, 3, 9, 7, 10, 2, 4, 5, 1] Zcount= 6264.228985093774
C= 263.1931358219208
Xcur= [1, 6, 8, 3, 9, 7, 10, 2, 4, 5, 1] Zcur= 6264.228985093774
i= 2
Xcount= [1, 6, 8, 5, 4, 2, 9, 7, 10, 3, 1] Zcount= 5053.486355007676
Z-Zcur= -1210.742630086098
Xcur= [1, 6, 8, 5, 4, 2, 9, 7, 10, 3, 1] Zcur= 5053.486355007676
i= 3
Xcount= [1, 6, 8, 3, 10, 7, 9, 2, 5, 4, 1] Zcount= 6080.020473177777
C= 1026.5341181701006
Xcur= [1, 6, 8, 3, 10, 7, 9, 2, 5, 4, 1] Zcur= 6080.020473177777
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
3.

Xcount= [1, 6, 8, 5, 4, 2, 9, 7, 10, 3, 1] Zcount= 5053.486355007676
C= 185.41969747260555
Xcur= [1, 6, 8, 5, 4, 2, 9, 7, 10, 3, 1] Zcur= 5053.486355007676
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
4.036792039871216
Tcur= 2225
i= 1
Xcount= [8, 6, 1, 3, 10, 7, 9, 2, 4, 5, 8] Zcount= 5053.486355007676
C= 0.0
Xcur= [8, 6, 1, 3, 10, 7, 9, 2, 4, 5, 8] Zcur= 5053.486355007676
i= 2
Xcount= [1, 6, 4, 2, 9, 7, 10, 3, 8, 5, 1] Zcount= 4716.417570140609
Z-Zcur= -337.0687848670668
Xcur= [1, 6, 4, 2, 9, 7, 10, 3, 8, 5, 1] Zcur= 4716.417570140609
i= 3
Xcount= [1, 6, 8, 3, 7, 10, 9, 2, 4, 5, 1] Zcount= 5569.293770880358
C= 852.876200739749
Xcur= [1, 6, 8, 3, 7, 10, 9, 2, 4, 5, 1] Zcur= 5569.293770880358
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
4.068725109100342
Tcur= 2220
i= 1
Xcount= [9, 7, 10, 3, 8, 6, 1, 2, 4, 5, 9] Zcount= 6437.1455369362575
C= 867.8517660558991
Xcur= [9, 7, 10, 3, 8, 6, 1, 2, 4, 5, 9] Zcur= 6437.1455369362575
i= 2
Xcount= [1, 6,

C= 780.5791178127502
Xcur= [1, 6, 8, 3, 10, 7, 9, 5, 4, 2, 1] Zcur= 6437.145536936257
i= 3
Xcount= [1, 6, 8, 3, 10, 5, 4, 2, 9, 7, 1] Zcount= 5677.61768517165
Z-Zcur= -759.5278517646066
Xcur= [1, 6, 8, 3, 10, 5, 4, 2, 9, 7, 1] Zcur= 5677.61768517165
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
4.242318153381348
Tcur= 2090
i= 1
Xcount= [2, 9, 7, 10, 3, 8, 6, 1, 4, 5, 2] Zcount= 6080.020473177777
C= 402.4027880061267
Xcur= [2, 9, 7, 10, 3, 8, 6, 1, 4, 5, 2] Zcur= 6080.020473177777
i= 2
Xcount= [1, 6, 8, 7, 10, 3, 9, 2, 4, 5, 1] Zcount= 5982.452027577094
Z-Zcur= -97.56844560068294
Xcur= [1, 6, 8, 7, 10, 3, 9, 2, 4, 5, 1] Zcur= 5982.452027577094
i= 3
Xcount= [1, 6, 8, 3, 5, 4, 2, 9, 7, 10, 1] Zcount= 5131.828610044533
Z-Zcur= -850.6234175325608
Xcur= [1, 6, 8, 3, 5, 4, 2, 9, 7, 10, 1] Zcur= 5131.828610044533
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 4685.807886934204
4.243380069732666
Tcur= 2085
i= 1
Xcount= [1, 6, 8, 3, 10, 7, 2, 9, 4, 5, 1] Zcount= 5656.56641

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,ganttColor,popupText,startElevMeters,endElevMeters,wayname,waycategory,surface,waytype,steepness,tollway
0,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,0.000000,42.989309,-78.807882,0,2.698756,...,darkgray,None,181.9,181.9,Manser Drive,No category,Asphalt,Street,0,None
1,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,2.698756,42.989382,-78.807787,0,7.400000,...,darkgray,None,181.9,181.6,Manser Drive,No category,Asphalt,Street,0,None
2,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,7.400000,42.989522,-78.807642,0,9.586844,...,darkgray,None,181.6,181.5,Homecrest Drive,No category,Asphalt,Street,0,None
3,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,9.586844,42.989524,-78.807754,0,25.342365,...,darkgray,None,181.5,181.6,Homecrest Drive,No category,Asphalt,Street,0,None
4,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,25.342365,42.989649,-78.808543,0,29.508100,...,darkgray,None,181.6,181.6,Homecrest Drive,No category,Asphalt,Street,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,6591.837439,42.989646,-78.804883,0,6593.900000,...,darkgray,None,180.0,179.8,Sweet Home Road,No category,Asphalt,Road,0,None
1702,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,6593.900000,42.989481,-78.805122,0,6595.813046,...,darkgray,None,179.8,179.7,Homecrest Drive,No category,Asphalt,Street,0,None
1703,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,6595.813046,42.989482,-78.805220,0,6643.100000,...,darkgray,None,179.7,180.0,Homecrest Drive,No category,Asphalt,Street,0,None
1704,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,6643.100000,42.989522,-78.807642,0,6647.801244,...,darkgray,None,180.0,180.1,Manser Drive,No category,Asphalt,Street,0,None


In [18]:
vrv.createLeaflet(arcs=X_best, nodes=nodesDF)

In [19]:
#%time
X_best=solveTSP_SA(nodesDF,distMeters,2)
X_best

Intial Solution : X0 = [1, 6, 8, 3, 10, 7, 9, 2, 4, 5, 1] Z0 = 55105.46037602928
Current Solution : Xcur = [1, 6, 8, 3, 4, 2, 9, 7, 10, 5, 1] Zcur = 56255.03457851153
Best known Solution : Xbest = [1, 6, 8, 3, 4, 2, 9, 7, 10, 5, 1] Zbest = 56255.03457851153
Tcur= 4000
i= 1
Xcount= [8, 6, 1, 3, 10, 7, 9, 2, 4, 5, 8] Zcount= 56477.62312894481
C= 222.58855043328367
Xcur= [8, 6, 1, 3, 10, 7, 9, 2, 4, 5, 8] Zcur= 56477.62312894481
i= 2
Xcount= [1, 6, 8, 3, 10, 7, 9, 4, 2, 5, 1] Zcount= 60803.69052548154
C= 4326.067396536731
Xcur= [1, 6, 8, 3, 10, 7, 9, 4, 2, 5, 1] Zcur= 60803.69052548154
i= 3
Xcount= [1, 2, 9, 7, 10, 3, 8, 6, 4, 5, 1] Zcount= 66655.232253435
C= 5851.541727953452
Xcur= [1, 2, 9, 7, 10, 3, 8, 6, 4, 5, 1] Zcur= 66655.232253435
Xbest= [1, 6, 8, 3, 4, 2, 9, 7, 10, 5, 1] Zbest= 56255.03457851153
0.0048182010650634766
Tcur= 3995
i= 1
Xcount= [1, 3, 8, 6, 10, 7, 9, 2, 4, 5, 1] Zcount= 61050.74738947708
Z-Zcur= -5604.484863957914
Xcur= [1, 3, 8, 6, 10, 7, 9, 2, 4, 5, 1] Zcur= 61050.

i= 3
Xcount= [4, 2, 9, 7, 10, 3, 8, 6, 1, 5, 4] Zcount= 55105.46037602928
Z-Zcur= -1372.1627529155303
Xcur= [4, 2, 9, 7, 10, 3, 8, 6, 1, 5, 4] Zcur= 55105.46037602928
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
0.24592828750610352
Tcur= 3895
i= 1
Xcount= [1, 6, 8, 3, 10, 7, 5, 4, 2, 9, 1] Zcount= 67067.40995602198
C= 11961.949579992695
i= 2
Xcount= [1, 6, 8, 3, 10, 7, 9, 2, 5, 4, 1] Zcount= 67950.13991877725
C= 12844.679542747966
i= 3
Xcount= [1, 8, 6, 3, 10, 7, 9, 2, 4, 5, 1] Zcount= 57651.77108826986
C= 2546.310712240578
Xcur= [1, 8, 6, 3, 10, 7, 9, 2, 4, 5, 1] Zcur= 57651.77108826986
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
0.24649810791015625
Tcur= 3890
i= 1
Xcount= [1, 6, 8, 3, 10, 7, 9, 5, 4, 2, 1] Zcount= 71941.35971120137
C= 14289.588622931507
i= 2
Xcount= [1, 6, 8, 3, 10, 5, 4, 2, 9, 7, 1] Zcount= 63452.89753787601
C= 5801.1264496061485
Xcur= [1, 6, 8, 3, 10, 5, 4, 2, 9, 7, 1] Zcur= 63452.89753787601
i= 3
Xcount= [1, 6, 8, 3, 10

Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
0.6946799755096436
Tcur= 3680
i= 1
Xcount= [1, 9, 7, 10, 3, 8, 6, 2, 4, 5, 1] Zcount= 71216.29559893493
C= 13863.121603871943
i= 2
Xcount= [1, 6, 8, 3, 10, 7, 9, 5, 4, 2, 1] Zcount= 71941.35971120137
C= 14588.185716138381
i= 3
Xcount= [10, 3, 8, 6, 1, 7, 9, 2, 4, 5, 10] Zcount= 63452.89753787601
C= 6099.7235428130225
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
0.695915937423706
Tcur= 3675
i= 1
Xcount= [1, 6, 8, 3, 7, 10, 9, 2, 4, 5, 1] Zcount= 62242.27248075415
C= 4889.098485691167
i= 2
Xcount= [1, 5, 4, 2, 9, 7, 10, 3, 8, 6, 1] Zcount= 55105.46037602929
Z-Zcur= -2247.713619033697
Xcur= [1, 5, 4, 2, 9, 7, 10, 3, 8, 6, 1] Zcur= 55105.46037602929
i= 3
Xcount= [1, 6, 8, 3, 10, 7, 9, 2, 5, 4, 1] Zcount= 67950.13991877725
C= 12844.67954274796
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
0.6964490413665771
Tcur= 3670
i= 1
Xcount= [1, 6, 8, 3, 5, 4, 2, 9, 7, 10, 1] Zcount= 57353.173

C= 10808.890598869511
i= 2
Xcount= [1, 6, 8, 3, 10, 7, 9, 2, 5, 4, 1] Zcount= 67950.13991877725
C= 11472.516789832436
i= 3
Xcount= [1, 6, 2, 9, 7, 10, 3, 8, 4, 5, 1] Zcount= 64930.520610657826
C= 8452.897481713015
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
0.9760410785675049
Tcur= 3455
i= 1
Xcount= [2, 9, 7, 10, 3, 8, 6, 1, 4, 5, 2] Zcount= 67950.13991877725
C= 11472.516789832436
i= 2
Xcount= [1, 4, 2, 9, 7, 10, 3, 8, 6, 5, 1] Zcount= 54434.974261981304
Z-Zcur= -2042.6488669635073
Xcur= [1, 4, 2, 9, 7, 10, 3, 8, 6, 5, 1] Zcur= 54434.974261981304
i= 3
Xcount= [1, 6, 8, 3, 10, 7, 9, 4, 2, 5, 1] Zcount= 60803.69052548154
C= 6368.716263500239
Xcur= [1, 6, 8, 3, 10, 7, 9, 4, 2, 5, 1] Zcur= 60803.69052548154
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
0.9780821800231934
Tcur= 3450
i= 1
Xcount= [1, 6, 8, 3, 10, 5, 4, 2, 9, 7, 1] Zcount= 63452.89753787601
C= 2649.207012394465
Xcur= [1, 6, 8, 3, 10, 5, 4, 2, 9, 7, 1] Zcur= 63452.89753787601
i= 2
Xc

Tcur= 3260
i= 1
Xcount= [1, 6, 8, 3, 9, 7, 10, 2, 4, 5, 1] Zcount= 70008.84913104733
C= 15603.471390509214
i= 2
Xcount= [1, 6, 8, 3, 2, 9, 7, 10, 4, 5, 1] Zcount= 67463.17275375279
C= 13057.795013214673
i= 3
Xcount= [1, 6, 5, 4, 2, 9, 7, 10, 3, 8, 1] Zcount= 54405.377740538126
C= 7.275957614183426e-12
Xcur= [1, 6, 5, 4, 2, 9, 7, 10, 3, 8, 1] Zcur= 54405.377740538126
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
1.1775431632995605
Tcur= 3255
i= 1
Xcount= [1, 6, 8, 3, 10, 7, 9, 4, 2, 5, 1] Zcount= 60803.69052548154
C= 6398.312784943417
i= 2
Xcount= [1, 6, 8, 3, 10, 7, 9, 5, 4, 2, 1] Zcount= 71941.35971120137
C= 17535.98197066324
Xcur= [1, 6, 8, 3, 10, 7, 9, 5, 4, 2, 1] Zcur= 71941.35971120137
i= 3
Xcount= [1, 6, 8, 9, 7, 10, 3, 2, 4, 5, 1] Zcount= 71137.69479591509
Z-Zcur= -803.664915286281
Xcur= [1, 6, 8, 9, 7, 10, 3, 2, 4, 5, 1] Zcur= 71137.69479591509
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
1.178694248199463
Tcur= 3250
i= 1
Xcount= [1, 3

Xcount= [1, 6, 8, 3, 10, 7, 9, 2, 5, 4, 1] Zcount= 67950.13991877725
C= 11472.516789832436
i= 3
Xcount= [1, 6, 8, 3, 10, 7, 2, 9, 4, 5, 1] Zcount= 63217.62917327935
C= 6740.0060443345355
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
1.440397024154663
Tcur= 3095
i= 1
Xcount= [1, 6, 8, 3, 10, 7, 9, 4, 2, 5, 1] Zcount= 60803.69052548154
C= 4326.067396536731
i= 2
Xcount= [1, 6, 8, 3, 9, 7, 10, 2, 4, 5, 1] Zcount= 70008.84913104733
C= 13531.226002102521
i= 3
Xcount= [1, 6, 8, 3, 10, 7, 9, 2, 5, 4, 1] Zcount= 67950.13991877725
C= 11472.516789832436
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
1.4433650970458984
Tcur= 3090
i= 1
Xcount= [1, 6, 8, 7, 10, 3, 9, 2, 4, 5, 1] Zcount= 66859.7176809786
C= 10382.094552033792
i= 2
Xcount= [1, 9, 7, 10, 3, 8, 6, 2, 4, 5, 1] Zcount= 71216.29559893493
C= 14738.672469990117
i= 3
Xcount= [1, 6, 8, 9, 7, 10, 3, 2, 4, 5, 1] Zcount= 71137.69479591509
C= 14660.071666970274
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbes

i= 3
Xcount= [1, 6, 4, 2, 9, 7, 10, 3, 8, 5, 1] Zcount= 52710.55175229228
Z-Zcur= -10675.790423158098
Xcur= [1, 6, 4, 2, 9, 7, 10, 3, 8, 5, 1] Zcur= 52710.55175229228
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
1.6386830806732178
Tcur= 2920
i= 1
Xcount= [1, 6, 8, 3, 10, 5, 4, 2, 9, 7, 1] Zcount= 63452.89753787601
C= 10742.345785583726
i= 2
Xcount= [1, 6, 8, 3, 10, 7, 9, 2, 5, 4, 1] Zcount= 67950.13991877725
C= 15239.588166484966
i= 3
Xcount= [1, 7, 10, 3, 8, 6, 9, 2, 4, 5, 1] Zcount= 69909.27180133312
C= 17198.720049040836
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
1.640491008758545
Tcur= 2915
i= 1
Xcount= [1, 6, 8, 3, 10, 9, 7, 2, 4, 5, 1] Zcount= 67286.51372781432
C= 14575.961975522041
i= 2
Xcount= [1, 6, 8, 9, 7, 10, 3, 2, 4, 5, 1] Zcount= 71137.69479591509
C= 18427.143043622804
i= 3
Xcount= [1, 6, 8, 7, 10, 3, 9, 2, 4, 5, 1] Zcount= 66859.7176809786
C= 14149.165928686321
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.4587830464

C= 12699.89780795978
i= 3
Xcount= [1, 6, 9, 7, 10, 3, 8, 2, 4, 5, 1] Zcount= 70360.82749531374
C= 15955.449754775625
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
1.9099171161651611
Tcur= 2725
i= 1
Xcount= [1, 6, 8, 3, 10, 2, 9, 7, 4, 5, 1] Zcount= 67626.65496088883
C= 13221.277220350712
i= 2
Xcount= [1, 6, 8, 3, 2, 9, 7, 10, 4, 5, 1] Zcount= 67463.17275375279
C= 13057.795013214673
i= 3
Xcount= [1, 6, 8, 3, 10, 7, 9, 2, 5, 4, 1] Zcount= 67950.13991877725
C= 13544.762178239129
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
1.9137771129608154
Tcur= 2720
i= 1
Xcount= [1, 6, 7, 10, 3, 8, 9, 2, 4, 5, 1] Zcount= 68473.7365007442
C= 14068.35876020608
i= 2
Xcount= [1, 6, 8, 5, 4, 2, 9, 7, 10, 3, 1] Zcount= 56477.62312894481
C= 2072.2453884066927
i= 3
Xcount= [1, 6, 8, 3, 10, 9, 7, 2, 4, 5, 1] Zcount= 67286.51372781432
C= 12881.135987276204
Xbest= [1, 6, 8, 4, 2, 9, 7, 10, 3, 5, 1] Zbest= 52368.45878304647
1.9149971008300781
Tcur= 2715
i= 1
Xcount= [3, 8

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,ganttColor,popupText,startElevMeters,endElevMeters,wayname,waycategory,surface,waytype,steepness,tollway
0,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,0.000000,42.989309,-78.807882,0,2.698756,...,darkgray,None,181.9,181.9,Manser Drive,No category,Asphalt,Street,0,None
1,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,2.698756,42.989382,-78.807787,0,7.400000,...,darkgray,None,181.9,181.6,Manser Drive,No category,Asphalt,Street,0,None
2,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,7.400000,42.989522,-78.807642,0,9.586844,...,darkgray,None,181.6,181.5,Homecrest Drive,No category,Asphalt,Street,0,None
3,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,9.586844,42.989524,-78.807754,0,25.342365,...,darkgray,None,181.5,181.6,Homecrest Drive,No category,Asphalt,Street,0,None
4,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,25.342365,42.989649,-78.808543,0,29.508100,...,darkgray,None,181.6,181.6,Homecrest Drive,No category,Asphalt,Street,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,6591.837439,42.989646,-78.804883,0,6593.900000,...,darkgray,None,180.0,179.8,Sweet Home Road,No category,Asphalt,Road,0,None
1702,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,6593.900000,42.989481,-78.805122,0,6595.813046,...,darkgray,None,179.8,179.7,Homecrest Drive,No category,Asphalt,Street,0,None
1703,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,6595.813046,42.989482,-78.805220,0,6643.100000,...,darkgray,None,179.7,180.0,Homecrest Drive,No category,Asphalt,Street,0,None
1704,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,6643.100000,42.989522,-78.807642,0,6647.801244,...,darkgray,None,180.0,180.1,Manser Drive,No category,Asphalt,Street,0,None


In [20]:
vrv.createLeaflet(arcs=X_best,nodes=nodesDF)